In [1]:
# !pip install interpret
# !pip install --user xgboost
# !pip install pytorch-tabnet
# !pip install anchor-exp

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from interpret import preserve, show
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

random_state=42

In [3]:
dataset_folder = 'dataset/explain_te-master/selected/'

file = 'diaretino.csv'
class_names = [0, 1]
_class = 'dr'

In [4]:
df = pd.read_csv(dataset_folder + file)
df.replace(('yes', 'no'), (1, 0), inplace=True)
df.head()

,qa,ps,ma0.5,ma0.6,ma0.7,ma0.8,ma0.9,ma1.0,ex0.5,ex0.6,ex0.7,ex0.8,ex0.9,ex1.0,exm1,exm2,eucmac,diaopt,amfm,dr
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1


In [5]:
X, y = df.drop(columns=[_class]), df[_class]

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=random_state)

In [7]:
x_test.shape

(346, 19)

# ML Model

In [8]:
import sklearn.metrics
import pandas as pd
import time
import numpy as np

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
random_state=42

In [11]:
x_train

,qa,ps,ma0.5,ma0.6,ma0.7,ma0.8,ma0.9,ma1.0,ex0.5,ex0.6,ex0.7,ex0.8,ex0.9,ex1.0,exm1,exm2,eucmac,diaopt,amfm
834,1,1,85,73,70,62,59,37,43.873596,17.321847,4.708070,1.229496,0.119951,0.015783,0.014205,0.012626,0.548025,0.107324,0
29,1,1,103,89,83,71,60,38,11.025085,3.762343,0.015592,0.010914,0.003118,0.003118,0.001559,0.001559,0.488566,0.134091,0
745,1,1,12,11,11,11,10,8,73.975723,39.190608,6.678279,0.779589,0.121811,0.014211,0.000000,0.000000,0.475377,0.117750,0
165,1,1,19,19,19,18,15,9,13.115054,5.845388,0.154330,0.050435,0.000000,0.000000,0.000000,0.000000,0.484477,0.109948,0
248,1,1,58,57,55,50,43,34,31.499303,16.223873,7.405699,0.924929,0.163038,0.083087,0.026650,0.010974,0.519667,0.125414,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,1,1,18,18,18,17,13,10,22.320425,10.157700,0.657133,0.044193,0.007686,0.003843,0.000000,0.000000,0.570065,0.090308,1
1095,1,1,34,34,34,34,33,24,136.285159,25.909910,15.509432,5.114110,0.307383,0.048480,0.000000,0.000000,0.523087,0.129967,1
1130,1,1,37,36,33,24,18,10,37.686807,4.978201,1.180336,0.184896,0.024986,0.007996,0.000000,0.000000,0.535271,0.093947,0
860,1,1,11,7,6,6,6,5,31.073428,14.277795,1.746907,0.010051,0.000000,0.000000,0.000000,0.000000,0.531741,0.133682,0


In [12]:
def classify_report(clfs, dataset):
    x_train, y_train, x_test, y_test = dataset
    data = []
    for clf, name, no_df in clfs:
        if no_df:
            x_tr, x_te = x_train, x_test
        else:
            x_tr, x_te = x_train.values,  x_test.values
        clf.fit(x_tr, y_train)
        pred = clf.predict(x_te)
        f1, acc = sklearn.metrics.f1_score(y_test, pred, average='binary'), sklearn.metrics.accuracy_score(y_test, pred)
        data.append([name, f1, acc])
    df = pd.DataFrame(data, columns = ['Name', 'F1', 'Acc.'])
    df = df.sort_values(by=['F1'])
    return df

In [13]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=random_state)

# clfs = [(rf, 'rf', True), (gbc, 'gbc', True), (_xgb, 'xgb', True), (ebm, 'ebm', True), (tbn, 'tbn', False)]
# clfs = [(rf, 'rf', True), (ebm, 'ebm', True), (tbn, 'tbn', False)]
clfs = [(rf, 'rf', True)]
dataset = x_train, y_train, x_test, y_test
classify_report(clfs, dataset)

,Name,F1,Acc.
0,rf,0.681948,0.679191


# XAI

In [14]:
class_names = ['NO','YES']
feature_names = x_train.columns.to_list()
# local = lime, shap, anchor, tabnet, ebm
# global = pfi, tabnet, ebm, shap
# methods = pfi, tabnet, ebm, shap, lime, anchor
# remaining = pdp, eli5, ice, adawhip, break down

### Anchors (Rules)

In [15]:
from anchor import utils
from anchor import anchor_tabular
import re

In [16]:
exp_anchor = anchor_tabular.AnchorTabularExplainer(
    class_names,
    feature_names,
    x_train.values[:,:],
    {})

In [17]:
len(x_test), (len(x_test) * 3/3)/(60*60)

(346, 0.0961111111111111)

In [18]:
import time, datetime
from joblib import Parallel, delayed
import itertools

n_jobs = 12 # 8 or 4 depending on how you want and RAM constraint (if it take too much RAM)

def anchor_explain_instance_step(i, lim, threshold):
    warnings.filterwarnings("ignore")
    out = []
    for k in range(i, lim):
        e = exp_anchor.explain_instance(x_test.iloc[k, :].values, rf.predict, threshold=threshold)
        out.append(e)
    return out

def prepare_anchor_explanations():
    anchor_explanations_index = {'threshold': {}}
    total = len(x_test)
    # total = 100
    for threshold in [0.9, 0.95, 0.99]:
        now = datetime.datetime.now()
        print('threshold', threshold, 'at', now)
        start = time.time()
        # anchor_explanations_index['threshold'][threshold] = [exp_anchor.explain_instance(x_test.iloc[i, :].values, rf.predict, threshold=threshold) for i in range(total)]
        # out = Parallel(n_jobs=n_jobs)(delayed(anchor_explain_instance_step)(i, (i) + n_jobs, threshold) for i in range(0, total, n_jobs))
        out = Parallel(n_jobs=n_jobs)(delayed(anchor_explain_instance_step)(i, min(total, i+int(total/n_jobs)), threshold) for i in range(0, total, int(total/n_jobs)))
        anchor_explanations_index['threshold'][threshold] = list(itertools.chain(*out))
        del out
        taken = time.time() - start
        print(f'> done; taken {taken:.2f} at throughput {taken/total:.2f}')
    anchor_explanations_index['pred'] = [exp_anchor.class_names[rf.predict([x_test.iloc[i, :].values])[0]] for i in range(total)] # it is basically original model's prediction
    anchor_explanations_index['total'] = total
    return anchor_explanations_index

try: del anchor_explanations_index; 
except: pass
anchor_explanations_index = prepare_anchor_explanations()

threshold 0.9 at 2024-06-27 15:29:01.493378
> done; taken 238.32 at throughput 0.69
threshold 0.95 at 2024-06-27 15:32:59.815614
> done; taken 298.35 at throughput 0.86
threshold 0.99 at 2024-06-27 15:37:58.160693
> done; taken 270.43 at throughput 0.78


In [19]:
def get_anchor_metrics(threshold=1.0):
    out = []
    for i in range(anchor_explanations_index['total']):
        exp = anchor_explanations_index['threshold'][threshold][i]
        out.append([exp.precision(), exp.coverage(), len(exp.features())])
    precison, coverage, simplicity = np.array(out).mean(axis=0)
    print('Threshold at', threshold)
    print(f' Precison: {precison:.2f}')
    print(f' Coverage: {coverage:.2f}')
    print(f' Simplicity: {simplicity:.2f}')

for thres in anchor_explanations_index['threshold']:
    get_anchor_metrics(threshold=thres)

Threshold at 0.9
 Precison: 0.91
 Coverage: 0.02
 Simplicity: 10.94
Threshold at 0.95
 Precison: 0.94
 Coverage: 0.01
 Simplicity: 13.86
Threshold at 0.99
 Precison: 0.96
 Coverage: 0.01
 Simplicity: 17.09


In [20]:
rf_predict_array = rf.predict(x_test)

In [21]:
rf_predict_array[1]

0

In [22]:
from sklearn.metrics import accuracy_score

achor_predict_array = np.array([{'YES': 1, 'NO': 0}[anchor_explanations_index['pred'][i]] for i in range(len(x_test))])  # this is orignal prediction of the model

print(f'Anchors Fidelity (Predict): {accuracy_score(rf_predict_array, achor_predict_array):.4f}')
print('Not Applicable')

Anchors Fidelity (Predict): 1.0000
Not Applicable


In [23]:
def add_noise(data, noise_level=0.01):
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

x_test_noisy = add_noise(x_test)
display(x_test_noisy)

,qa,ps,ma0.5,ma0.6,ma0.7,ma0.8,ma0.9,ma1.0,ex0.5,ex0.6,ex0.7,ex0.8,ex0.9,ex1.0,exm1,exm2,eucmac,diaopt,amfm
174,0.997741,0.004625,12.997616,13.002578,12.993689,11.997647,10.999002,5.982943,31.009847,14.299925,0.982623,0.044320,-0.012731,-0.001226,-0.002871,-0.000952,0.552065,0.126327,0.002949
752,0.986028,0.999480,56.999969,55.994833,54.006373,48.999225,46.990486,34.002444,61.389638,29.785762,6.220250,0.414256,0.005896,-0.004679,0.017810,0.004387,0.548798,0.099691,0.002351
467,1.003259,1.008497,59.976711,58.993549,57.005325,55.001216,48.996581,33.002600,50.264627,35.391393,13.506311,4.250885,1.956105,0.373764,0.056174,-0.005053,0.520719,0.125535,0.007998
240,1.008484,0.995352,47.988565,47.980621,48.001434,47.991439,41.000420,30.008154,34.045263,13.427156,6.037846,1.510342,0.336784,-0.004826,-0.005134,0.001465,0.554921,0.108877,0.014372
667,1.001202,0.004637,27.008496,26.998858,27.007785,25.005976,23.000602,19.009984,10.696945,4.555066,1.396258,0.075865,0.002611,-0.010422,0.003070,0.016111,0.500542,0.098900,0.984271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,1.018309,1.004071,17.004640,16.998269,17.014771,13.993635,12.010432,9.018432,68.675685,23.971749,8.881368,1.348017,0.144590,0.032096,0.011685,-0.010527,0.510141,0.100334,0.995137
1054,1.000871,0.989527,17.009321,16.993016,16.000673,14.007225,10.014279,10.013429,236.219656,107.781603,65.611268,14.594933,0.077658,-0.003868,-0.006482,-0.002967,0.484545,0.094517,0.987806
961,0.997695,0.994729,55.000230,53.997687,52.990506,52.993310,52.013096,46.012068,42.826698,22.436569,4.271256,0.475457,0.010645,-0.001741,0.009430,-0.001822,0.509127,0.072527,0.009564
617,1.008354,0.017868,62.009676,62.011219,61.004615,54.997952,52.010167,45.023836,43.409036,28.479187,11.751549,2.076581,0.179672,0.012926,-0.007641,-0.007293,0.475810,0.117850,-0.012813


In [24]:
import time, datetime
from joblib import Parallel, delayed
import itertools

def anchor_explain_instance_step_noisy(i, lim, threshold):
    warnings.filterwarnings("ignore")
    out = []
    for k in range(i, lim):
        e = exp_anchor.explain_instance(x_test_noisy.iloc[k, :].values, rf.predict, threshold=threshold)
        out.append(e)
    return out

def prepare_anchor_explanations_noisy():
    anchor_explanations_index = {'threshold': {}}
    total = len(x_test_noisy)
    # total = 100
    for threshold in [0.9, 0.95, 0.99]:
        now = datetime.datetime.now()
        print('threshold', threshold, 'at', now)
        start = time.time()
        out = Parallel(n_jobs=n_jobs)(delayed(anchor_explain_instance_step_noisy)(i, min(total, i+int(total/n_jobs)), threshold) for i in range(0, total, int(total/n_jobs)))
        anchor_explanations_index['threshold'][threshold] = list(itertools.chain(*out))
        del out
        taken = time.time() - start
        print(f'> done; taken {taken:.2f} at throughput {taken/total:.2f}')
    anchor_explanations_index['total'] = total
    return anchor_explanations_index

try: del anchor_explanations_index_noisy; 
except: pass
anchor_explanations_index_noisy = prepare_anchor_explanations_noisy()

threshold 0.9 at 2024-06-27 15:42:35.654281
> done; taken 228.25 at throughput 0.66
threshold 0.95 at 2024-06-27 15:46:23.907785
> done; taken 256.71 at throughput 0.74
threshold 0.99 at 2024-06-27 15:50:40.613149
> done; taken 312.03 at throughput 0.90


In [25]:
def get_features_in_rules(exp_anchors):
    features = []
    for anchor_name in exp_anchors:
        idx = re.search('<|>|=', anchor_name).start()
        features.append(anchor_name[:idx-1])
    return features

def change_in_features(f1, f2):
    s1 = set(f1)
    s2 = set(f2)
    return 1- (len(s1.intersection(s2))/len(s1.union(s2)))


def calculate_robustness(explainer, total):
    for threshold in anchor_explanations_index['threshold']:
        robustness_list = []
        for i in range(total):
            exp = anchor_explanations_index['threshold'][threshold][i]
            noisy_exp = anchor_explanations_index_noisy['threshold'][threshold][i]
            delta = change_in_features(get_features_in_rules(exp.names()), get_features_in_rules(noisy_exp.names()))
            # print(delta)
            # print(get_features_in_rules(exp.names()), get_features_in_rules(noisy_exp.names()))
            robustness_list.append(delta)
        print('Threshold at', threshold, np.mean(robustness_list) if robustness_list else 0)
    return 

calculate_robustness(exp_anchor, len(x_test[:]))

Threshold at 0.9 0.5808541686383618
Threshold at 0.95 0.5731657315182588
Threshold at 0.99 0.571715559054558


## Rule based explanations, alternate binary cases

In [26]:
ids_to_explain_list = [9, 10]

for _id in ids_to_explain_list:
    print('ID', _id,  'Prediction: ', exp_anchor.class_names[rf.predict([x_test.iloc[_id, :].values])[0]])
    np.random.seed(random_state)

    # Original explanation cases
    # Noisy explanation to compare how the rule (features in the rule) changes due to insertion of Nois
    for _indexes in [('Original', x_test), ('Noisy', x_test_noisy)]:
        msg, dataset = _indexes
        print('>', msg)
        exp = exp_anchor.explain_instance(dataset.iloc[_id, :].values, rf.predict, threshold=0.99)
        print('  >> Anchor: %s' % (' AND '.join(exp.names())))
        print('  >> Precision: %.2f' % exp.precision())
        print('  >> Coverage: %.2f' % exp.coverage())

ID 9 Prediction:  YES
> Original
  >> Anchor: ma0.5 > 35.00 AND 7.70 < ex0.6 <= 16.83 AND eucmac <= 0.50 AND ma0.6 > 34.00 AND ma0.9 <= 43.00 AND ex0.5 > 22.03 AND 0.08 < ex0.8 <= 0.47 AND ex0.9 <= 0.02 AND ma1.0 <= 32.00 AND ex0.7 > 1.22 AND ps <= 1.00 AND qa <= 1.00 AND amfm <= 1.00 AND ma0.8 <= 48.00
  >> Precision: 0.97
  >> Coverage: 0.00
> Noisy
  >> Anchor: 22.03 < ex0.5 <= 88.45 AND ma0.5 <= 55.00 AND 0.00 < ex1.0 <= 0.03 AND 1.22 < ex0.7 <= 4.41 AND 0.00 < ex0.9 <= 0.02 AND ma0.6 <= 52.00 AND 7.70 < ex0.6 <= 31.71 AND diaopt > 0.10 AND ma1.0 > 17.00 AND ma0.8 > 29.00 AND 0.08 < ex0.8 <= 0.47 AND eucmac > 0.50
  >> Precision: 0.86
  >> Coverage: 0.00
ID 10 Prediction:  YES
> Original
  >> Anchor: ma0.5 > 35.00 AND ex0.5 > 88.45 AND ma0.6 > 34.00 AND amfm > 0.00
  >> Precision: 1.00
  >> Coverage: 0.05
> Noisy
  >> Anchor: ma0.5 > 35.00 AND exm2 > 0.00 AND ma0.6 > 34.00 AND amfm > 0.00 AND ex0.5 > 43.72 AND ma1.0 <= 32.00
  >> Precision: 1.00
  >> Coverage: 0.04
